## Desafío - Regresión desde el aprendizaje de máquinas

* Para poder realizar este desafío debes haber revisado la lectura y videos correspondiente a la unidad.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminado el desafío, comprime la carpeta y sube el .zip


#### Contexto
En esta sesión trabajaremos una base de datos sobre los precios de las viviendas en Boston,
utilizada en el paper Harrison Jr, D., & Rubinfeld, D. L. (1978). Hedonic housing prices and the
demand for clean air. Journal of environmental economics and management, 5(1), 81-102.
Nuestro objetivo es desarrollar un modelo predictivo para el valor mediano de las casas mediante el
entrenamiento de un modelo de regresión lineal.


* crim : Tasa de criminalidad por sector de Boston
* zn proporción de terreno residencial asignado para terrenos baldíos.
* indus proporción de negocios no asociados al comercio por sector.
* chas Dummy. 1 si el sector colinda con el río Charles, 0 de lo contrario.
* nox Concentración de dióxido de carbono
* rm cantidad promedio de habitaciones por casa.
* age proporción de casas construídas antes de 1940
* dis distancia promedio a cinco centros de empleos.
* rad índice de accesibilidad a autopistas.
* tax nivel de impuestos asociados a viviendas.
* ptratio razón alumno:profesor por sector de Boston.
* black proporción de afroamericanos por sector de Boston.
* lstat porcentaje de población de estratos bajos.
* medv valor mediano de las casas

### Desafío 1: Prepare el ambiente de trabajo

* Importe las librerías básicas para el análisis de datos.
* Importe el módulo linear_model , y las funciones mean_squared_error , r2_score y train_test_split.
* Importe la base de datos boston.csv y elimine la columna Unnamed: 0.
* Obtenga las medidas descriptivas de la base de datos con .describe().

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model

df = pd.read_csv('boston.csv')
df.drop(df.columns[0], axis = 1, inplace = True)
df.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


### Desafío 2: División de la muestra

* Genere conjuntos de entrenamiento y validación con train_test_split.
* Genere una segmentación del 33% para las muestras de validación.
* Incluya una semilla pseudoaleatoria

In [2]:
y_vec = df.loc[:, 'medv']
X_mat = df.loc[:, 'crim':'lstat']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_mat, y_vec, test_size=0.33, random_state=11238)

### Desafío 3: Generación de modelos
Ahora implementaremos dos versiones del modelo lineal:
* Con intercepto y atributos normalizados.
* Sin intercepto y atributos no normalizados.


* Cada versión debe generarse en un nuevo objeto inicializado.
* Posteriormente se deben entrenar los modelos especificando la matriz y vector de entrenamiento.
* Con los modelos entrenados, genere una predicción con su matriz de validación.

In [4]:
# con intercepto y atributos normalizados
model_1 = linear_model.LinearRegression(fit_intercept=True, normalize=True)

# sin intercepto y sin normalizar
model_2 = linear_model.LinearRegression(fit_intercept=False, normalize=False)

In [5]:
# Entrenamos los modelos
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train);

In [6]:
# Predicción
model_1_yhat = model_1.predict(X_test)
model_2_yhat = model_2.predict(X_test)

### Desafío 4: Obtención de métricas
* Ahora generemos una función llamada report_scores que ingrese como argumentos el vector de datos predichos y el vector de datos por validar.

* La función debe imprimir las métricas del Error Cuadrático Promedio y R2. 

* Reporte las métricas para ambos modelos. En base a ello, seleccione el mejor modelo.

In [7]:
def report_scores(prediction, validation):
    print('MSE: {}'.format(mean_squared_error(prediction, validation)))
    print('R2: {}'.format(r2_score(prediction, validation)))
    
report_scores(y_test, model_1_yhat)
report_scores(y_test, model_2_yhat)

MSE: 30.69783151774075
R2: 0.6005199127106761
MSE: 34.269399961452756
R2: 0.5540420215010212


En base a esto, es mejor el **primero** (con intercepto y normalizado) ya que el R2 es mayor y el MSE es menor

### Desafío 5: Refactorización del modelo
* Genere una función llamada fetch_features que ingrese como argumentos la base de datos (df) y el nombre del vector objetivo. El nombre del vector debe ser medv por defecto
* La función debe retornar una lista con las correlaciones entre cada atributo y el vector objetivo y su nombre.
* Reporte brevemente cuales los 6 atributos con una mayor correlación con medv

In [8]:
def fetch_features(dataframe, vector_name = 'medv'):
    correlations = dataframe.corr()
    
    vectors = []
    corr = []

    for col in dataframe:
        if col != vector_name:
            vectors.append(col)
            corr.append(correlations.loc[col, vector_name])
    
    return pd.DataFrame(data=corr, index=vectors, columns=[vector_name])

features = fetch_features(df)
features

,medv
crim,-0.388305
zn,0.360445
indus,-0.483725
chas,0.175260
nox,-0.427321
rm,0.695360
age,-0.376955
dis,0.249929
rad,-0.381626
tax,-0.468536


In [9]:
print('Los mejores son:')
best = abs(features).sort_values('medv').tail(6)
best

Los mejores son:


,medv
nox,0.427321
tax,0.468536
indus,0.483725
ptratio,0.507787
rm,0.695360
lstat,0.737663


### Desafío 6: Refactorización del modelo predictivo
* Genere otros conjuntos de entrenamiento y validación en base a una matriz con los 6 atributos identificados y el vector objetivo.
* Entrene un modelo en base al mejor desempeño.
* Reporte las métricas para el nuevo modelo.

In [10]:
y_vec = df.loc[:, 'medv']
X_mat = df[['lstat', 'ptratio', 'rm', 'indus', 'tax', 'nox']]
X_train, X_test, y_train, y_test = train_test_split(X_mat, y_vec, test_size=0.33, random_state=11238)


model_1 = linear_model.LinearRegression(fit_intercept=True, normalize=True)
model_2 = linear_model.LinearRegression(fit_intercept=False, normalize=False)

model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)

model_1_yhat = model_1.predict(X_test)
model_2_yhat = model_2.predict(X_test)

report_scores(y_test, model_1_yhat)
report_scores(y_test, model_2_yhat)

MSE: 37.519164674969346
R2: 0.5117518587357747
MSE: 39.11190606242991
R2: 0.4910250374251438


En este caso, el mejor modelo es el que está normalizado (model_1) y con intercepto, porque el MSE es menor y el R2 es mayor

### Desafío 7: Predicción de casos
A continuación se generaron dos arrays que representan el peor escenario posible
( worst_neighbor ) y el mejor escenario posible ( best_neighbor ).

Ingrese los arrays en el modelo entrenado y reporte cuál sería el valor esperado dada las
condiciones

worst_neighbor = np.array([37.9, 12.6, 3.5, 27.7, 187, 0.87]).reshape(1, -1)


best_neighbor = np.array([1.73, 22, 8.7, 0.46, 711, 0.38]).reshape(1, -1)


Los valores númericos representan los siguientes atributos ['lstat', 'ptratio', 'rm', 'indus', 'tax', 'nox'].


In [11]:
worst_neighbor = np.array([37.9, 12.6, 3.5, 27.7, 187, 0.87]).reshape(1, -1)
best_neighbor = np.array([1.73, 22, 8.7, 0.46, 711, 0.38]).reshape(1, -1)

In [12]:
print('Para el peor, el valor esperado es {}'.format(model_1.predict(worst_neighbor)[0]))
print('Para el mejor, el valor esperado es {}'.format(model_1.predict(best_neighbor)[0]))

Para el peor, el valor esperado es 3.015722292338504
Para el mejor, el valor esperado es 35.31255604600602
